<a href="https://colab.research.google.com/github/timsetsfire/wandb-examples/blob/main/colab/wandb_with_scala_via_wandb_java_client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WandB with Scala

This notebook is meant to be run in google colab

In [ ]:
%%sh
curl https://pjreddie.com/media/files/mnist_train.csv > mnist_train.csv
curl https://pjreddie.com/media/files/mnist_test.csv > mnist_test.csv
pip install wandb[grpc]==0.10.32 -q --upgrade
git clone https://github.com/wandb/client-java.git
wandb login e5643b9c64e9d3343ca45748ff06b8a6267738bb
apt-get install maven &> /dev/null
cd client-java 

In [ ]:
%%sh
cd client-java
make install
make build

## Install the Scala Kernel
If you get a "scala" kernel not recognized warning when loading up the notebook for the first time, start by running the two cells below. Once you are done **reload the page** to load the notebook in the installed Scala kernel.

In [ ]:
%%shell
SCALA_VERSION=2.12.8 ALMOND_VERSION=0.3.0+16-548dc10f-SNAPSHOT
curl -Lo coursier https://git.io/coursier-cli
chmod +x coursier
./coursier bootstrap \
    -r jitpack -r sonatype:snapshots \
    -i user -I user:sh.almond:scala-kernel-api_$SCALA_VERSION:$ALMOND_VERSION \
    sh.almond:scala-kernel_$SCALA_VERSION:$ALMOND_VERSION \
    --sources --default=true \
    -o almond-snapshot --embed-files=false &> /dev/null
rm coursier
./almond-snapshot --install --global --force &> /dev/null
rm almond-snapshot

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 42577  100 42577    0     0  65604      0 --:--:-- --:--:-- --:--:-- 65604


In [ ]:
%%shell
echo "{
  \"language\" : \"scala\",
  \"display_name\" : \"Scala\",
  \"argv\" : [
    \"bash\",
    \"-c\",
    \"env LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libpython3.6m.so:\$LD_PRELOAD java -jar /usr/local/share/jupyter/kernels/scala/launcher.jar --connection-file {connection_file}\"
  ]
}" > /usr/local/share/jupyter/kernels/scala/kernel.json

## Refresh your browser at this point

In [ ]:
val clientJar = "/content/client-java/target/client-ng-java-1.0-SNAPSHOT-jar-with-dependencies.jar"
val path = java.nio.file.FileSystems.getDefault().getPath(clientJar)
val x = ammonite.ops.Path(path)
interp.load.cp(x)

clientJar: String = "/content/client-java/target/client-ng-java-1.0-SNAPSHOT-jar-with-dependencies.jar"
path: java.nio.file.Path = /content/client-java/target/client-ng-java-1.0-SNAPSHOT-jar-with-dependencies.jar
x: os.Path = root/'content/"client-java"/'target/"client-ng-java-1.0-SNAPSHOT-jar-with-dependencies.jar"

## Download Necessary Libraries

Using import as below we download all libraries necessary to create the MNist Model.  This may take a while.

In [ ]:
import $ivy.`org.nd4j:nd4j-native-platform:1.0.0-M2`
import $ivy.`org.deeplearning4j:deeplearning4j-datasets:1.0.0-M2`
import $ivy.`org.deeplearning4j:deeplearning4j-core:1.0.0-M2`
import $ivy.`org.nd4j:nd4s_2.11:1.0.0-beta7`
import $ivy.`org.nd4j:nd4j-api:1.0.0-M2`
import $ivy.`org.nd4j:nd4j-native-platform:1.0.0-M2`
// import $ivy.`org.datavec:datavec-api:1.0.0-M2`
// import $ivy.`org.datavec:datavec-data-image:1.0.0-M2`
// import $ivy.`org.datavec:datavec-local:1.0.0-M2`
// import $ivy.`org.deeplearning4j:deeplearning4j-datasets:1.0.0-M2`

In [ ]:
  import org.deeplearning4j.nn.conf.MultiLayerConfiguration;
  import org.deeplearning4j.nn.conf.NeuralNetConfiguration;
  import org.deeplearning4j.nn.conf.layers.DenseLayer;
  import org.deeplearning4j.nn.conf.layers.OutputLayer;
  import org.deeplearning4j.nn.multilayer.MultiLayerNetwork;
  import org.deeplearning4j.nn.weights.WeightInit;
  import org.deeplearning4j.optimize.listeners.ScoreIterationListener;
  import org.nd4j.evaluation.classification.Evaluation;
  import org.nd4j.linalg.activations.Activation;
  import org.nd4j.linalg.dataset.api.iterator.DataSetIterator;
  import org.nd4j.linalg.learning.config.{Nadam, Adam};
  import org.nd4j.linalg.lossfunctions.LossFunctions.LossFunction;
  import org.nd4j.linalg.dataset.DataSet;
  import org.nd4j.linalg.factory.Nd4j
  import java.io.{File, PrintWriter}
  import org.datavec.api.records.reader.RecordReader;
  import org.datavec.api.records.reader.impl.csv.CSVRecordReader;
  import org.datavec.api.split.FileSplit;
  import org.deeplearning4j.datasets.datavec.RecordReaderDataSetIterator;


import org.deeplearning4j.nn.conf.MultiLayerConfiguration;
  
import org.deeplearning4j.nn.conf.NeuralNetConfiguration;
  
import org.deeplearning4j.nn.conf.layers.DenseLayer;
  
import org.deeplearning4j.nn.conf.layers.OutputLayer;
  
import org.deeplearning4j.nn.multilayer.MultiLayerNetwork;
  
import org.deeplearning4j.nn.weights.WeightInit;
  
import org.deeplearning4j.optimize.listeners.ScoreIterationListener;
  
import org.nd4j.evaluation.classification.Evaluation;
  
import org.nd4j.linalg.activations.Activation;
  
import org.nd4j.linalg.dataset.api.iterator.DataSetIterator;
  
import org.nd4j.linalg.learning.config.{Nadam, Adam};
  
import org.nd4j.linalg.lossfunctions.LossFunctions.LossFunction;
  
import org.nd4j.linalg.dataset.DataSet;
  
import org.nd4j.linalg.factory.Nd4j
  
import java.io.{File, PrintWriter}
  
import org.datavec.api.records.reader.RecordReader;
  
import org.datavec.api.records.reader.impl.csv.CSVRecordReader;
  
import org.datavec.api.split.FileSplit;


In [ ]:
def loadData(path: String, batchSize: Int, labelIndex: Int, numClasses: Int): DataSetIterator = {
    val file = new java.io.File(path)
    val reader: RecordReader = new CSVRecordReader()
    reader.initialize(new FileSplit( file ))
    new RecordReaderDataSetIterator(reader, batchSize, labelIndex, numClasses); //reader, batch size, label index, num of label classes
}
{{
  val trainIter = loadData("mnist_train.csv", 256, 0, 10)
  val testIter = loadData("mnist_test.csv", 256, 0, 10)
}}

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


defined function loadData

In [ ]:
def loadData2(path: String, labelIndex: Int, numClasses: Int) = {
  val data = Nd4j.readNumpy(path, ",")
  val labels = data.getColumn(labelIndex)
  val Array(m,n) = data.shape
  val features = data.getColumns(  (0 until n.toInt).filter( _ != labelIndex):_*)
  val oneHotLabels = Nd4j.zeros(m,numClasses) 
  labels.dup.data.asInt.zipWithIndex.foreach { 
    case (k,v) => oneHotLabels.putScalar(v,k,1.0)
  }
  val dataset = new DataSet(features, oneHotLabels)
  dataset
}
// wrap in braces to suppress all stdout

val train = loadData2("mnist_train.csv", 0, 10)
val test = loadData2("mnist_test.csv", 0, 10)

defined function loadData2
train: DataSet = ===========INPUT===================
[[         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0], 
  ..., 
 [         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0]]
=================OUTPUT==================
[[         0,         0,         0,  ...         0,         0,         0], 
 [    1.0000,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0], 
  ..., 
 [         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,         0,  ...         0,         0,         0], 
 [         0,         0,   

In [ ]:
import com.wandb.grpc.WandbServer

import com.wandb.grpc.WandbServer

In [ ]:
import com.wandb.client

import com.wandb.client

In [ ]:
import com.wandb

import com.wandb

In [ ]:
import sys.process._

import sys.process._

In [ ]:
import java.io.File

import java.io.File

In [ ]:
import java.io.File
import sys.process._
val model_dir = new File("./my-model")
s"wandb artifact get --root ${model_dir} --type model tim-w/MNIST-v4/model-2ryu9yvh:v2" !
model_dir.listFiles

wandb: Downloading model artifact tim-w/MNIST-v4/model-2ryu9yvh:v2
wandb: Artifact downloaded to ./my-model


import java.io.File

import sys.process._


model_dir: File = ./my-model
res0_3: Int = 0
res0_4: Array[File] = Array(./my-model/model.ckpt)

res29: Array[File] = Array(./my-model/model.ckpt)

In [ ]:
import java.io.File
val out = "wandb artifact get --type model tim-w/MNIST-v4/model-2ryu9yvh:v2" !
val model_dir = new File("artifacts")
model_dir.listFiles

wandb: Downloading model artifact tim-w/MNIST-v4/model-2ryu9yvh:v2
wandb: Artifact downloaded to ./artifacts/model-2ryu9yvh:v2


import java.io.File

out: Int = 0
model_dir: File = artifacts
res21_3: Array[File] = Array(artifacts/model-2ryu9yvh:v2)

In [ ]:
"ls -r ./my-model" !!

res17: String = """model.ckpt
"""

In [ ]:
import scala.collection.JavaConverters._
import com.wandb.client._
import org.json.JSONObject;


val rate=0.0015
val numEpochs=1
val numColumns=28
val randomSeed=123
val numRows=28
val outputNum=10

val config = new JSONObject()

val configMap = Map(
  "numRows" -> numRows, "numColumns" -> numColumns, "outputNum" -> outputNum,
  "randomSeed" -> randomSeed, "numEpochs" -> numEpochs, "rate" -> rate
  )

configMap.foreach{ case (k,v) => config.put(k,v) }

val tags = List("scala", "dl4j", "client-java").asJava

import scala.collection.JavaConverters._

import com.wandb.client._

import org.json.JSONObject;



rate: Double = 0.0015
numEpochs: Int = 1
numColumns: Int = 28
randomSeed: Int = 123
numRows: Int = 28
outputNum: Int = 10
config: JSONObject = {"randomSeed":123,"numRows":28,"rate":0.0015,"numEpochs":1,"outputNum":10,"numColumns":28}
configMap: Map[String, AnyVal] = Map(
  "rate" -> 0.0015,
  "numEpochs" -> 1,
  "numColumns" -> 28,
  "randomSeed" -> 123,
  "numRows" -> 28,
  "outputNum" -> 10
)
tags: java.util.List[String] = SeqWrapper(List("scala", "dl4j", "client-java"))

In [ ]:
val modelConf = new NeuralNetConfiguration.Builder()
            .seed(randomSeed) //include a random seed for reproducibility
            .activation(Activation.RELU)
            .weightInit(WeightInit.XAVIER)
            .updater(new Adam())
            .l2(rate * 0.0001) // regularize learning model
            .list()
            .layer(new DenseLayer.Builder() //create the first input layer.
                    .nIn(numRows * numColumns)
                    .nOut(40)
                    .build())
            .layer(new DenseLayer.Builder() //create the second input layer
                    .nIn(40).activation(Activation.IDENTITY)
                    .nOut(10)
                    .build())
            .layer(new OutputLayer.Builder(LossFunction.MCXENT) //create hidden layer
                    .activation(Activation.SOFTMAX)
                    .nOut(outputNum)
                    .build())
            .build();

modelConf: MultiLayerConfiguration = {
  "backpropType" : "Standard",
  "cacheMode" : "NONE",
  "confs" : [ {
    "cacheMode" : "NONE",
    "dataType" : "FLOAT",
    "epochCount" : 0,
    "iterationCount" : 0,
    "layer" : {
      "@class" : "org.deeplearning4j.nn.conf.layers.DenseLayer",
      "activationFn" : {
        "@class" : "org.nd4j.linalg.activations.impl.ActivationReLU",
        "max" : null,
        "negativeSlope" : null,
        "threshold" : null
      },
      "biasInit" : 0.0,
      "biasUpdater" : null,
      "constraints" : null,
      "gainInit" : 1.0,
      "gradientNormalization" : "None",
      "gradientNormalizationThreshold" : 1.0,
      "hasBias" : true,
      "hasLayerNorm" : false,
      "idropout" : null,
      "iupdater" : {
        "@class" : "org.nd4j.linalg.learning.config.Adam",
        "beta1" : 0.9,
        "beta2" : 0.999,
        "epsilon" : 1.0E-8,
        "learningRate" : 0.001
      },
      "layerName" : "layer0",
      "nin" : 784,
      "nou

In [ ]:
val model = new MultiLayerNetwork(modelConf);

model: MultiLayerNetwork = org.deeplearning4j.nn.multilayer.MultiLayerNetwork@1f100c45

In [ ]:

import org.deeplearning4j.nn.api.Model;
import org.nd4j.linalg.dataset.DataSet;

import org.deeplearning4j.optimize.api.BaseTrainingListener;
import java.io.Serializable;
import com.wandb.client._
import org.json.JSONObject;

case class WandBListener(logIteration: Int = 10,
                         testDataset: DataSet,
                         run: WandbRun) extends BaseTrainingListener with Serializable {

  if(logIteration <= 0) throw new Exception(s"Iteration must be greater than 0")

  override def iterationDone(model: Model, iteration: Int, epoch: Int): Unit = { 
    if(iteration % logIteration == 0) {
      val trainingScore = model.score();
      val testScore = model.asInstanceOf[MultiLayerNetwork].score(testDataset)
      val data = new JSONObject() 
      data.put("epoch", epoch)
      data.put("iteration", iteration)
      data.put("train_loss", trainingScore)
      data.put("test_loss", testScore)
      run.log(data)
      println(s"Score on train dataset at iteration $iteration is $trainingScore")
      println(s"Score on test dataset at iteration $iteration is $testScore")
    }
  }
}

import org.deeplearning4j.optimize.api.BaseTrainingListener;

import java.io.Serializable;

import org.deeplearning4j.nn.api.Model;

import org.nd4j.linalg.dataset.DataSet;


defined class WandBListener

In [ ]:
val runBuilder = new WandbRun.Builder()
runBuilder.withConfig(config).withProject("dl4j-wandb-java-client").setTags(tags).setJobType("training")
val run = runBuilder.build
model.setListeners(new WandBListener(model, 10, test, run));  
model.init();

(1 to 500).foreach{
 i => model.fit(train)
 if(i % 50 == 0){
    val testLogLoss = model.score(test)
    val trainingLogLoss = model.score(train)
 }
}


runBuilder: WandbRun.Builder = com.wandb.client.WandbRun$Builder@6a5e946f
res9_1: WandbRun.Builder = com.wandb.client.WandbRun$Builder@6a5e946f
run: WandbRun = com.wandb.client.WandbRun@56c66058

In [ ]:
run.getClass.getMethods

res16: Array[java.lang.reflect.Method] = Array(
  public void com.wandb.client.WandbRun.finish(),
  public void com.wandb.client.WandbRun.finish(int),
  public static void com.wandb.client.WandbRun.main(java.lang.String[]) throws java.io.IOException,java.lang.InterruptedException,
  public com.wandb.grpc.WandbServer$HistoryResult com.wandb.client.WandbRun.log(org.json.JSONObject,int),
  public com.wandb.grpc.WandbServer$HistoryResult com.wandb.client.WandbRun.log(org.json.JSONObject),
  public com.wandb.grpc.WandbServer$RunRecord com.wandb.client.WandbRun.data(),
  public void com.wandb.client.WandbRun.printRunInfo(),
  public final native void java.lang.Object.wait(long) throws java.lang.InterruptedException,
  public final void java.lang.Object.wait(long,int) throws java.lang.InterruptedException,
  public final void java.lang.Object.wait() throws java.lang.InterruptedException,
  public boolean java.lang.Object.equals(java.lang.Object),
  public java.lang.String java.lang.Object.toS

In [ ]:
run.getClass.getMethods

res12: Array[java.lang.reflect.Method] = Array(
  public void com.wandb.client.WandbRun.finish(),
  public void com.wandb.client.WandbRun.finish(int),
  public static void com.wandb.client.WandbRun.main(java.lang.String[]) throws java.io.IOException,java.lang.InterruptedException,
  public com.wandb.grpc.WandbServer$HistoryResult com.wandb.client.WandbRun.log(org.json.JSONObject,int),
  public com.wandb.grpc.WandbServer$HistoryResult com.wandb.client.WandbRun.log(org.json.JSONObject),
  public com.wandb.grpc.WandbServer$RunRecord com.wandb.client.WandbRun.data(),
  public void com.wandb.client.WandbRun.printRunInfo(),
  public final native void java.lang.Object.wait(long) throws java.lang.InterruptedException,
  public final void java.lang.Object.wait(long,int) throws java.lang.InterruptedException,
  public final void java.lang.Object.wait() throws java.lang.InterruptedException,
  public boolean java.lang.Object.equals(java.lang.Object),
  public java.lang.String java.lang.Object.toS

In [ ]:
model.setListeners(new WandBListener(model, 10, test, run));  //print the score with every iteration
// model.setListeners(new WandBListener(50, test, wandb));  //print the score with every iteration
model.init();

In [ ]:
val output = model.output(train.getFeatures);
val eval = new Evaluation(10);
eval.eval(train.getLabels, output);

output: org.nd4j.linalg.api.ndarray.INDArray = [[         0,         0,1.3599e-13,  ...4.0409e-31,4.5754e-18,         0], 
 [8.4078e-45,         0,1.3994e-37,  ...         0,    0.9905,         0], 
 [         0,         0,         0,  ...         0,         0,         0], 
  ..., 
 [         0,         0,    1.0000,  ...         0, 9.9249e-7,         0], 
 [         0,         0,         0,  ...         0,    1.0000,         0], 
 [4.1927e-31,7.0069e-27,         0,  ...1.1962e-29,         0, 3.876e-21]]
eval: Evaluation = 

========================Evaluation Metrics========================
 # of classes:    10
 Accuracy:        0.0860
 Precision:       0.0800
 Recall:          0.0879
 F1 Score:        0.0547
Precision, recall & F1: macro-averaged (equally weighted avg. of 10 classes)


=========================Confusion Matrix=========================
    0    1    2    3    4    5    6    7    8    9
---------------------------------------------------
   58    0 1391   25 2679  218  

In [ ]:
(1 to 500).foreach{
 i => model.fit(train)
 if(i % 50 == 0){
    val testLogLoss = model.score(test)
    val trainingLogLoss = model.score(train)
 }
}



In [ ]:
run.shutdownNow()

cmd20.sc:1: value shutdownNow is not a member of com.wandb.client.WandbRun
val res20 = run.shutdownNow()
                ^Compilation Failed

: ignored

In [ ]:
run.finish()

: ignored